In [14]:
#!pip install -r requirements.txt
import numpy as np 
import pandas as pd


In [46]:
f = "data/tcga_brca_all_clean.csv.zip"

## Preprocessing genomic data 

In [47]:
n_bins = 4
df = pd.read_csv(f,compression='zip')

df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
_,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

# adapt bins 
bins[0] = 0 
bins[-1] = np.inf
#
labels=[i for i in range(n_bins)]
labels

df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels))
df.head(5)

,case_id,slide_id,site,is_female,oncotree_code,age,survival_months_discretized,survival_months,censorship,train,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
1,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
2,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
3,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-962...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
4,TCGA-3C-AALK,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,3C,1.0,IDC,52.0,2,47.57,1.0,1.0,...,-0.2431,-0.3657,-0.4472,-1.0557,0.8232,-1.0342,1.0729,-1.4634,-0.5710,-0.5402


scale genomic data first and store in DF again
then add labels to stratify 

In [28]:
# todo add new column k-th-fold that contains an int in arange(k) where k is a cluster 
groundtruth = df["survival_months_discretized"]
censorship = df["censorship"]
genomics = df[df.keys()[10:]]
df = df.sample(frac=1,random_state=1)
df.head()


,case_id,slide_id,site,is_female,oncotree_code,age,survival_months_discretized,survival_months,censorship,train,...,ZSWIM6,ZSWIM8,ZXDA,ZXDB,ZXDC,ZYG11A,ZYX,ZZEF1,ZZZ3,snoU13
329,TCGA-D1-A102,TCGA-D1-A102-01Z-00-DX1.13029F09-CF07-49B5-B5C...,D1,1,UEC,49.0,0,4.43,1,1.0,...,0,0,0,0,0,0,0,0,0,0
344,TCGA-D1-A16E,TCGA-D1-A16E-01Z-00-DX1.C5BE7CE0-B192-4E7C-AC9...,D1,1,UEC,73.0,3,36.76,1,1.0,...,0,0,0,0,0,1,0,0,0,0
464,TCGA-EY-A215,TCGA-EY-A215-01Z-00-DX1.E64C6134-1E9B-4777-A42...,EY,1,UEC,60.0,1,18.86,1,1.0,...,1,1,0,1,0,0,1,1,1,0
30,TCGA-A5-A0VQ,TCGA-A5-A0VQ-01Z-00-DX1.DE40A1AA-70AF-4884-BAB...,A5,1,UEC,62.0,1,15.93,1,1.0,...,0,0,0,0,0,1,0,0,0,0
132,TCGA-AX-A060,TCGA-AX-A060-01Z-00-DX2.24E305A1-4F4D-42DF-ABB...,AX,1,UEC,77.0,3,35.32,1,1.0,...,0,0,0,0,0,0,0,0,0,0


## k-Fold crossvalidation
1. Cluster dataset into k groups (shuffle first(seeded) then stratify by patient,split into equal chunks)
2. use one group for testing and the others for training
3. rotate such that each group is once used for testing
4. use distribution of testing results to estimate real value 



In [13]:
k = 7 
a = np.arange(k) # df k-th fold 
for i in range(k):  
    a = (a+1)%k
    print(f"train on {a[:-1]} test on {a[-1]}, store results in foldername: experiment1_fold{a[-1]}")



train on [1 2 3 4 5 6] test on 0, store results in foldername: experiment1_fold0
train on [2 3 4 5 6 0] test on 1, store results in foldername: experiment1_fold1
train on [3 4 5 6 0 1] test on 2, store results in foldername: experiment1_fold2
train on [4 5 6 0 1 2] test on 3, store results in foldername: experiment1_fold3
train on [5 6 0 1 2 3] test on 4, store results in foldername: experiment1_fold4
train on [6 0 1 2 3 4] test on 5, store results in foldername: experiment1_fold5
train on [0 1 2 3 4 5] test on 6, store results in foldername: experiment1_fold6


In [71]:
B = 100 

col1 = np.arange(B)
col1
col2 = np.random.rand(B)
df = pd.DataFrame({"ids":col1,"data":col2})
df = df.sample(frac=1)


k = 7-1# 7 folds 
df["kth-fold"]= df.index//(len(df)//k)
df.value_counts

,ids,data,kth-fold
8,8,0.876328,0
66,66,0.592062,4
96,96,0.487023,6
73,73,0.016899,4
15,15,0.091167,0
...,...,...,...
76,76,0.962324,4
28,28,0.338014,1
36,36,0.475050,2
14,14,0.385757,0


In [64]:
df.index

Index([59, 87, 37, 83, 58, 34,  2, 22, 79, 10, 44, 16, 50, 84, 28, 35,  1, 42,
        0, 24, 41,  7, 82, 36, 73, 71, 86, 46,  9, 67, 57, 70, 38, 80, 76, 56,
       62, 69, 99, 30, 63, 75, 61, 90, 33, 29, 45,  3, 89, 17,  6, 32, 55, 39,
       91, 96, 27, 94, 12,  5, 68, 52, 95, 97, 14, 13, 54, 43, 53, 77, 47, 81,
       26, 65, 21, 88, 72,  8, 19,  4, 11, 51, 15, 40, 48, 93, 60, 25, 98, 64,
       20, 18, 66, 78, 31, 23, 49, 74, 92, 85],
      dtype='int64')

In [44]:
len(df)//7

14

In [49]:
df

,case_id,slide_id,site,is_female,oncotree_code,age,survival_months_discretized,survival_months,censorship,train,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
1,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
2,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
3,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-962...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
4,TCGA-3C-AALK,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,3C,1.0,IDC,52.0,2,47.57,1.0,1.0,...,-0.2431,-0.3657,-0.4472,-1.0557,0.8232,-1.0342,1.0729,-1.4634,-0.5710,-0.5402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,TCGA-WT-AB44,TCGA-WT-AB44-01Z-00-DX1.B6ECEA7C-DA26-4B34-88C...,WT,1.0,ILC,77.0,1,29.01,1.0,1.0,...,-1.1686,-1.5424,-1.0868,-1.4608,-0.0141,-2.0773,3.9959,-1.4221,-1.0560,1.7134
1019,TCGA-XX-A899,TCGA-XX-A899-01Z-00-DX1.08FE27B7-73B8-4CE3-ACF...,XX,1.0,ILC,46.0,0,15.34,1.0,1.0,...,-0.6444,0.2435,0.2311,0.7384,-0.8560,-0.7456,0.7535,0.9765,-0.6290,-0.4313
1020,TCGA-XX-A89A,TCGA-XX-A89A-01Z-00-DX1.671E2AD6-4D1A-4579-88C...,XX,1.0,ILC,68.0,0,16.03,1.0,1.0,...,-0.1942,-0.6109,-0.7641,1.2241,0.7029,-0.5363,0.9538,-0.0743,-0.0954,0.0371
1021,TCGA-Z7-A8R5,TCGA-Z7-A8R5-01Z-00-DX1.3BDB407F-514C-4131-B05...,Z7,1.0,ILC,61.0,3,107.98,1.0,1.0,...,-1.0210,-1.2416,-1.0705,-1.3571,-0.5215,-1.3964,1.6214,-1.5199,-0.4771,0.0497
